<h1> aWhere API </h1>

In [1]:
from __future__ import division
import pandas as pd
import datetime
from aWhere import Awhere
%pylab inline
pylab.rcParams['figure.figsize'] = (12.0, 8.0) # set size of figures"
plt.rcParams.update({'font.size': 24})

DATE_FORMAT = '%Y-%m-%d'

Populating the interactive namespace from numpy and matplotlib


In [2]:
awhere = Awhere()

<h1> import information from woreda mapped file </h1>

In [45]:
#region_df = pd.read_csv('/Users/attiladobi/zenysis/pipeline/auto/ethiopia/bin/shared/data/woreda_mapped.csv')
woreda_df = pd.read_csv('/Users/attiladobi/zen_staging/elevation_area/woreda_info.csv')

In [38]:
START_DATE = '2013-01-01'
LAST_DATE = (datetime.datetime.now() - datetime.timedelta(days=1))
LAST_DATE_STR = LAST_DATE.strftime(DATE_FORMAT)

N_COMPLETE = 741

for lat, lon, geoKey in woreda_df[['WoredaLat', 'WoredaLon', 'GeoKey']].values[N_COMPLETE:]:
    # Check it lat/lon is valid
    if lat !=0:
        # Reset the start date for every Woreda
        start_date = datetime.datetime.strptime(START_DATE, DATE_FORMAT)
        start_date_str = start_date.strftime(DATE_FORMAT)
        results_df = []

        while start_date < LAST_DATE : 
            result = awhere.single_call(lat, lon, start_date_str, LAST_DATE_STR)
            result_df = pd.DataFrame(awhere.flatten_single(result))
            start_date = datetime.datetime.strptime(result_df.iloc[-1]['date'], DATE_FORMAT) + datetime.timedelta(days=1)
            start_date_str = start_date.strftime(DATE_FORMAT)
            results_df.append(result_df)

        dataframe = pd.concat(results_df, ignore_index=True)
        dataframe['GeoKey']  = geoKey
        dataframe.to_csv('weather/%s.csv' % geoKey.replace(' ','_'))

https://api.awhere.com/v2/weather/locations/9.616357,42.750231/observations/2013-01-01,2017-03-08/?limit=120
https://api.awhere.com/v2/weather/locations/9.616357,42.750231/observations/2013-05-01,2017-03-08/?limit=120
https://api.awhere.com/v2/weather/locations/9.616357,42.750231/observations/2013-08-29,2017-03-08/?limit=120
https://api.awhere.com/v2/weather/locations/9.616357,42.750231/observations/2013-12-27,2017-03-08/?limit=120
https://api.awhere.com/v2/weather/locations/9.616357,42.750231/observations/2014-04-26,2017-03-08/?limit=120
https://api.awhere.com/v2/weather/locations/9.616357,42.750231/observations/2014-08-24,2017-03-08/?limit=120
https://api.awhere.com/v2/weather/locations/9.616357,42.750231/observations/2014-12-22,2017-03-08/?limit=120
https://api.awhere.com/v2/weather/locations/9.616357,42.750231/observations/2015-04-21,2017-03-08/?limit=120
https://api.awhere.com/v2/weather/locations/9.616357,42.750231/observations/2015-08-19,2017-03-08/?limit=120
https://api.awhere.

In [6]:
#dataframe.to_csv('weather/weather_%s_%s_%s.csv' % (START_DATE, LAST_DATE_STR, geoKey.replace(' ','_')))

<h1> Lookup file with geoKey from geoKey.replace(' ','_') </h1>

In [39]:
region_df[['WoredaLat', 'WoredaLon', 'GeoKey']].values[-1]

array([13.756371601067402, 37.3954474200696,
       'tigray__west tigray__welkayit__'], dtype=object)

In [46]:
woreda_df.iloc[10]['WoredaLat']

'11.315658431000134'